In [1]:
!pip install nltk
!pip install rouge-score
!pip install transformers
!pip install bert-score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=95f300684f4ce1ba9b830fdfe9a2b30bdf3f35c2c9815dd58f2eb54478ebdb45
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 2.9 MB/s eta 0:00:00


In [2]:
from nltk.tokenize import word_tokenize
import nltk
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.meteor_score import meteor_score
from rouge_score import rouge_scorer
from bert_score import BERTScorer
import pprint
import math

# Ensure required NLTK data is available
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('punkt_tab')


def scoreCalculator(referenceText,hypothesisText):
    if type(hypothesisText) != str:
      print("Empty string entered")
      hypothesisText = ""
    # Preprocess: Tokenize for BLEU and METEOR scores
    hypothesisText_tokens = word_tokenize(hypothesisText)
    referenceText_tokens = word_tokenize(referenceText)

    # Get BLEU score
    BLEU = sentence_bleu([referenceText_tokens], hypothesisText_tokens)

    # Get METEOR score
    METEOR = meteor_score([referenceText_tokens], hypothesisText_tokens)

    # Get ROUGE score
    ROUGEscorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    ROUGE = ROUGEscorer.score(referenceText, hypothesisText)

    # Get BERT score
    BERTscorer = BERTScorer(model_type='bert-base-uncased', lang="en", rescale_with_baseline=True)
    BERT_P, BERT_R, BERT_F1 = BERTscorer.score([hypothesisText], [referenceText])

    # Return dict and print
    scoreDict = {
        "BLEU": round(BLEU, 2),
        "METEOR": round(METEOR, 2),
        "ROUGE1_precision": round(ROUGE["rouge1"].precision, 2),
        "ROUGE1_recall": round(ROUGE["rouge1"].recall, 2),
        "ROUGE1_F1": round(ROUGE["rouge1"].fmeasure, 2),
        "ROUGE2_precision": round(ROUGE["rouge2"].precision, 2),
        "ROUGE2_recall": round(ROUGE["rouge2"].recall, 2),
        "ROUGE2_F1": round(ROUGE["rouge2"].fmeasure, 2),
        "ROUGEL_precision": round(ROUGE["rougeL"].precision, 2),
        "ROUGEL_recall": round(ROUGE["rougeL"].recall, 2),
        "ROUGEL_F1": round(ROUGE["rougeL"].fmeasure, 2),
        "BERT_precision": round(BERT_P.item(), 2),
        "BERT_recall": round(BERT_R.item(), 2),
        "BERT_F1": round(BERT_F1.item(), 2),
    }

    print("Score Summary:")
    for key, value in scoreDict.items():
      print(f"{key}: {value}")

    return scoreDict


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [ ]:
import pandas as pd

ES_hypothesis_df = pd.read_csv("summarization_BART_ES.csv")
ES_reference_df = pd.read_csv("csvFiles/papers_EStoEN.csv")

JP_hypothesis_df = pd.read_csv("summarization_BART_JP.csv")
JP_reference_df = pd.read_csv("csvFiles/papers_JPtoEN.csv")

In [ ]:
ES_referenceText = ES_reference_df['contribution_translated']
JP_referenceText = JP_reference_df['contribution_translated']

ES_hypothesis_BART = ES_hypothesis_df['BART']
ES_hypothesis_BART_ft = ES_hypothesis_df['BART_ft']

JP_hypothesis_BART = JP_hypothesis_df['BART']
JP_hypothesis_BART_ft = JP_hypothesis_df['BART_ft']

In [ ]:
def evaluator(referenceSeries,hypothesisSeries):
  assert len(referenceSeries) == len(hypothesisSeries)

  dfSeed = []
  for i in range(len(referenceSeries)):
    scores = scoreCalculator(referenceSeries[i],hypothesisSeries[i])
    dfSeed.append(scores)

  df = pd.DataFrame(dfSeed)

  return df

In [ ]:
df_JP_BART = evaluator(JP_referenceText,JP_hypothesis_BART)
df_JP_BART_ft = evaluator(JP_referenceText,JP_hypothesis_BART_ft)

df_ES_BART = evaluator(ES_referenceText,ES_hypothesis_BART)
df_ES_BART_ft = evaluator(ES_referenceText,ES_hypothesis_BART_ft)


Score Summary:
BLEU: 0.1
METEOR: 0.3
ROUGE1_precision: 0.65
ROUGE1_recall: 0.38
ROUGE1_F1: 0.48
ROUGE2_precision: 0.31
ROUGE2_recall: 0.18
ROUGE2_F1: 0.22
ROUGEL_precision: 0.49
ROUGEL_recall: 0.29
ROUGEL_F1: 0.36
BERT_precision: 0.48
BERT_recall: 0.44
BERT_F1: 0.46


/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


Score Summary:
BLEU: 0.0
METEOR: 0.26
ROUGE1_precision: 0.25
ROUGE1_recall: 0.43
ROUGE1_F1: 0.32
ROUGE2_precision: 0.08
ROUGE2_recall: 0.14
ROUGE2_F1: 0.1
ROUGEL_precision: 0.18
ROUGEL_recall: 0.3
ROUGEL_F1: 0.22
BERT_precision: 0.27
BERT_recall: 0.39
BERT_F1: 0.33


/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


Score Summary:
BLEU: 0.0
METEOR: 0.12
ROUGE1_precision: 0.08
ROUGE1_recall: 0.3
ROUGE1_F1: 0.12
ROUGE2_precision: 0.01
ROUGE2_recall: 0.05
ROUGE2_F1: 0.02
ROUGEL_precision: 0.06
ROUGEL_recall: 0.26
ROUGEL_F1: 0.1
BERT_precision: 0.16
BERT_recall: 0.23
BERT_F1: 0.19


...

Score Summary:
BLEU: 0.0
METEOR: 0.21
ROUGE1_precision: 0.39
ROUGE1_recall: 0.27
ROUGE1_F1: 0.32
ROUGE2_precision: 0.1
ROUGE2_recall: 0.07
ROUGE2_F1: 0.08
ROUGEL_precision: 0.26
ROUGEL_recall: 0.18
ROUGEL_F1: 0.21
BERT_precision: 0.42
BERT_recall: 0.36
BERT_F1: 0.39


/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


Score Summary:
BLEU: 0.0
METEOR: 0.14
ROUGE1_precision: 0.44
ROUGE1_recall: 0.23
ROUGE1_F1: 0.3
ROUGE2_precision: 0.03
ROUGE2_recall: 0.02
ROUGE2_F1: 0.02
ROUGEL_precision: 0.22
ROUGEL_recall: 0.12
ROUGEL_F1: 0.15
BERT_precision: 0.34
BERT_recall: 0.32
BERT_F1: 0.33
Score Summary:
BLEU: 0.17
METEOR: 0.45
ROUGE1_precision: 0.38
ROUGE1_recall: 0.62
ROUGE1_F1: 0.47
ROUGE2_precision: 0.21
ROUGE2_recall: 0.35
ROUGE2_F1: 0.26
ROUGEL_precision: 0.28
ROUGEL_recall: 0.46
ROUGEL_F1: 0.34
BERT_precision: 0.44
BERT_recall: 0.54
BERT_F1: 0.49


/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


Score Summary:
BLEU: 0.0
METEOR: 0.19
ROUGE1_precision: 0.42
ROUGE1_recall: 0.22
ROUGE1_F1: 0.29
ROUGE2_precision: 0.08
ROUGE2_recall: 0.04
ROUGE2_F1: 0.05
ROUGEL_precision: 0.19
ROUGEL_recall: 0.1
ROUGEL_F1: 0.13
BERT_precision: 0.34
BERT_recall: 0.18
BERT_F1: 0.25


In [ ]:
df_JP_BART.to_csv("scores_JP_BART.csv")
df_JP_BART_ft.to_csv("scores_JP_BART_ft.csv")
df_ES_BART.to_csv("scores_ES_BART.csv")
df_ES_BART_ft.to_csv("scores_ES_BART_ft.csv")
